In [10]:
import dai
import pandas as pd

import sys
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/General_Tools")
sys.path.append("/home/aiuser/work/GitHub/BigQuant_Resources_Collection/BigQuant_Research_Frameworks/Strategy_Tools")
from General_Tools import *
from Stock_Factor_Strategy_Tools import *

In [11]:
sd = get_date_str('2019-12-31')
ed = get_date_str('2026-02-13')

In [12]:
sql_dict = {
    "factor1": {
        "sql": """
        SELECT
            date,
            instrument,
            m_CORR(close/m_lag(close, 1), log(volume/m_lag(volume, 1)+1), 20) AS _factor_temp,
            -1 * _factor_temp AS factor
        FROM cn_stock_bar1d
        ORDER BY date, instrument
        """,
        "weight": 0.1
    },
    "factor2": {
        "sql": """
        SELECT
            date,
            instrument,
            m_CORR(close/m_lag(close, 1), log(volume/m_lag(volume, 1)+1), 30) AS _factor_temp,
            -1 * _factor_temp AS factor
        FROM cn_stock_bar1d
        ORDER BY date, instrument
        """,
        "weight": 0.1
    },
    "factor3": {
        "sql": """
        SELECT
            date,
            instrument,
            m_CORR(close/m_lag(close, 1), log(volume/m_lag(volume, 1)+1), 60) AS _factor_temp,
            -1 * _factor_temp AS factor
        FROM cn_stock_bar1d
        ORDER BY date, instrument
        """,
        "weight": 0.8
    },
}


In [13]:
sql_trade = get_trade_sql(
    process_factor_sql(synthesize_multi_factor_sql(sql_dict)), 
    number_security=5, 
    position_allocation="equal", 
    rebalance_period="day"
    )

In [14]:
strategy_df = dai.query(sql_trade, filters = {"date":[sd, ed]}).df()
strategy_df

,date,instrument,score,score_rank,position
0,2020-04-02,000963.SZ,3.398935,1.0,0.2
1,2020-04-02,002696.SZ,3.128448,2.0,0.2
2,2020-04-02,000620.SZ,3.089624,3.0,0.2
3,2020-04-02,601579.SH,3.061477,4.0,0.2
4,2020-04-02,601158.SH,3.038146,5.0,0.2
...,...,...,...,...,...
7125,2026-02-13,600941.SH,3.193694,1.0,0.2
7126,2026-02-13,688496.SH,3.188013,2.0,0.2
7127,2026-02-13,600476.SH,3.073625,3.0,0.2
7128,2026-02-13,600986.SH,3.024134,4.0,0.2


In [15]:
backtest_factor_strategy(strategy_df, capital_base=500000)

[2026-02-17 16:07:15] [info     ] bigtrader.v34 开始运行 ..
[2026-02-17 16:07:15] [info     ] 2020-04-02, 2026-02-13, , , instruments=717
[2026-02-17 16:07:16] [info     ] bigtrader module V2.2.0
[2026-02-17 16:07:16] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-17 16:07:27] [info     ] backtest done, raw_perf_ds:dai.DataSource("_09b3fda7d2004c6780d6f137a2581e8d")


[2026-02-17 16:07:29] [info     ] bigtrader.v34 运行完成 [13.427s].


,date,trading_days,portfolio_value,starting_value,ending_value,starting_cash,ending_cash,returns,algorithm_period_return,benchmark_period_return,...,today_sell_balance,commission,capital_changed,orders,transactions,positions,POS_FAC,TRA_FAC,LOG,benchmark_returns
2020-04-02,2020-04-02,1,5.000082e+05,0.00,0.00,500008.229492,500008.229492,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,"[{'id': '', 'order_time': '15:00:00', 'dt': 20...",[],[],{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.000000
2020-04-03,2020-04-03,2,4.998910e+05,0.00,390797.00,500008.229492,109093.989492,-0.000234,-0.000234,-0.005707,...,0.0,117.24,0.0,"[{'symbol': '000963.SZ', 'id': '202004020001',...","[{'symbol': '000963.SZ', 'dt': 2020-04-03 15:0...","[{'symbol': '000963.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'ERROR', 'msg': 'order[08:30:00][BU...",-0.005707
2020-04-07,2020-04-07,3,5.071705e+05,390797.00,398671.00,109093.989492,108499.459492,0.014562,0.014324,0.017001,...,1525.0,11.53,0.0,"[{'symbol': '002696.SZ', 'id': '202004030001',...","[{'symbol': '000620.SZ', 'dt': 2020-04-07 15:0...","[{'symbol': '000963.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.022838
2020-04-08,2020-04-08,4,5.094480e+05,398671.00,350315.00,108499.459492,159132.959492,0.004491,0.018879,0.012268,...,197360.0,300.50,0.0,"[{'symbol': '000620.SZ', 'id': '202004070001',...","[{'symbol': '000620.SZ', 'dt': 2020-04-08 15:0...","[{'symbol': '000963.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'ERROR', 'msg': 'order[08:30:00][BU...",-0.004654
2020-04-09,2020-04-09,5,5.205655e+05,350315.00,474843.00,159132.959492,45722.549492,0.021823,0.041114,0.015606,...,0.0,40.41,0.0,"[{'symbol': '601579.SH', 'id': '202004080001',...","[{'symbol': '601579.SH', 'dt': 2020-04-09 15:0...","[{'symbol': '000963.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.003298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-09,2026-02-09,1422,1.557374e+06,1502764.24,1556914.92,40344.499492,459.509492,0.009245,2.114698,0.263628,...,272144.0,452.99,0.0,"[{'symbol': '600986.SH', 'id': '202602060001',...","[{'symbol': '600986.SH', 'dt': 2026-02-09 15:0...","[{'symbol': '002707.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.016250
2026-02-10,2026-02-10,1423,1.553170e+06,1556914.92,1552548.71,459.509492,621.249492,-0.002700,2.106289,0.265031,...,310002.0,506.26,0.0,"[{'symbol': '301359.SZ', 'id': '202602090001',...","[{'symbol': '301359.SZ', 'dt': 2026-02-10 15:0...","[{'symbol': '002707.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.001110
2026-02-11,2026-02-11,1424,1.539445e+06,1552548.71,1516324.22,621.249492,23120.659492,-0.008837,2.078839,0.262225,...,309652.0,492.59,0.0,"[{'symbol': '600643.SH', 'id': '202602100001',...","[{'symbol': '600643.SH', 'dt': 2026-02-11 15:0...","[{'symbol': '002707.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",-0.002218
2026-02-12,2026-02-12,1425,1.514548e+06,1516324.22,1475030.08,23120.659492,39518.079492,-0.016173,2.029046,0.263767,...,293920.0,474.58,0.0,"[{'symbol': '600986.SH', 'id': '202602110001',...","[{'symbol': '600986.SH', 'dt': 2026-02-12 15:0...","[{'symbol': '002707.SZ', 'direction': '1', 'cu...",{},{'LOG': []},"[{'level': 'INFO', 'msg': 'order[15:00:00][id:...",0.001222
